# Azure-Synapse-Retail-Recommender-Solution-Accelerator

_This accelerator is built to provide developers to quickly build a Retail Recommender Solution based on Azure Synapse Analytics Platform._

![image-alt-text](https://d33wubrfki0l68.cloudfront.net/0d75c87ae627b0413623ca5d07c6be5add5a7bb1/052bd/assets/images/posts/carts.jpg)

**Important - Do not use in production, for demonstration purposes only - please review the legal notices before continuing**

In [111]:
# Check synapse version
import sys
print(sys.version)

StatementMeta(, , , Waiting, )

3.6.11 | packaged by conda-forge | (default, Aug  5 2020, 20:04:48) 
[GCC 7.5.0]

In [112]:
# loading required packages
import matplotlib.pyplot as plt
from datetime import datetime
from dateutil import parser
from pyspark.sql.functions import unix_timestamp

from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *

from pyspark.ml import Pipeline
from pyspark.ml import PipelineModel
from pyspark.ml.feature import RFormula
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorIndexer
from pyspark.ml.classification import LogisticRegression
from pyspark.mllib.evaluation import BinaryClassificationMetrics
from pyspark.ml.evaluation import BinaryClassificationEvaluator

StatementMeta(, , , Waiting, )

### Data Ingestion
###### Read Spark table as a Spark dataframe
Loading Order, Customer and Product data from WWIRetailLakeDB

In [113]:
%%pyspark
spark.sql("show databases").show()
sqlContext.sql("show tables in WWImportersContosoRetailLakeDB").show()
df_order = spark.sql("SELECT * FROM WWImportersContosoRetailLakeDB.WWImportersContosoOrder")
df_order = df_order.drop("ItemSku", "EntityCode", "Brand")
print("WWImportersContosoOrder =======================================================")
print(df_order.columns)
print(df_order.count())

df_brand = spark.sql("SELECT * FROM WWImportersContosoRetailLakeDB.BrandsList")
print("BrandList =======================================================")
print(df_brand.columns)
print(df_brand.count())

df_customer = spark.sql("SELECT * FROM WWImportersContosoRetailLakeDB.WWImporterContosoCustomers")
df_customer = df_customer.drop("EntityCode", "OrderId")
print("WWImporterContosoCustomers =======================================================")
print(df_customer.columns)
print(df_customer.count())

df_product = spark.sql("SELECT * FROM WWImportersContosoRetailLakeDB.ProductsList")
print("ProductList =======================================================")
print(df_product.columns)
print(df_product.count())

StatementMeta(, , , Waiting, )

+--------------------+
|        databaseName|
+--------------------+
|             default|
|WWImportersContos...|
+--------------------+

+--------------------+--------------------+-----------+
|            database|           tableName|isTemporary|
+--------------------+--------------------+-----------+
|wwimporterscontos...|WWImportersContos...|      false|
|wwimporterscontos...|WWImporterContoso...|      false|
|wwimporterscontos...|        ProductsList|      false|
|wwimporterscontos...|          BrandsList|      false|
+--------------------+--------------------+-----------+

WWImportersContosoOrder =======================================================
['OrderId', 'CustomerId', 'OrderTotalInvoicedAmount', 'ProductSalesPriceAmount', 'ProductListPriceAmount', 'TotalShippingChargeAmount', 'OrderTotalTaxAmount', 'OrderStatus', 'Quantity', 'OrderDate', 'ProductAdjustmentAmount', 'ProductAdjustmentPercentage', 'ProductName', 'ProductId']
8263
BrandList ================================

In [114]:
# Top 5 most bought products
product_count = df_product.groupBy('ProductId').count()
product_count.show(5)

StatementMeta(, , , Waiting, )

+---------+-----+
|ProductId|count|
+---------+-----+
|      125|    1|
|        7|    1|
|       51|    1|
|      124|    1|
|       15|    1|
+---------+-----+
only showing top 5 rows

In [115]:
# top 5 brand
top_brand = df_brand[df_brand["EntityCode"] == 'WideWorldImporters'].groupBy('BrandName').count().sort('count', ascending=False).limit(5) # only keep top 5 categories
#top_category = top_category.withColumnRenamed("category_code","category_code_tmp")
display(top_brand)

StatementMeta(, , , Waiting, )

SynapseWidget(Synapse.DataFrame, 818611ee-6547-4817-a4cf-ae01eb14e062)

In [116]:
# Combining customer, order and product data
df = df_order.join(df_product, on='ProductId')
df = df.join(df_customer, on='CustomerId')

print("Columns: ")
print(df.columns)
print("No. of rows:", df.count())

Columns: 
['CustomerId', 'ProductId', 'OrderId', 'OrderTotalInvoicedAmount', 'ProductSalesPriceAmount', 'ProductListPriceAmount', 'TotalShippingChargeAmount', 'OrderTotalTaxAmount', 'OrderStatus', 'Quantity', 'OrderDate', 'ProductAdjustmentAmount', 'ProductAdjustmentPercentage', 'ProductName', 'ItemSku', 'ProductCategories', 'ProductName', 'ProductDescription', 'ProductShortDescription', 'BrandName', 'EntityCode', 'CustomerFirstName', 'CustomerMiddleName', 'CustomerLastName', 'CustomerGender', 'CustomerEmailAddress', 'CustomerFullName', 'CustomerDOB', 'City', 'Country']

StatementMeta(, , , Waiting, )

Columns: 
['CustomerId', 'ProductId', 'OrderId', 'OrderTotalInvoicedAmount', 'ProductSalesPriceAmount', 'ProductListPriceAmount', 'TotalShippingChargeAmount', 'OrderTotalTaxAmount', 'OrderStatus', 'Quantity', 'OrderDate', 'ProductAdjustmentAmount', 'ProductAdjustmentPercentage', 'ProductName', 'ItemSku', 'ProductCategories', 'ProductName', 'ProductDescription', 'ProductShortDescription', 'BrandName', 'EntityCode', 'CustomerFirstName', 'CustomerMiddleName', 'CustomerLastName', 'CustomerGender', 'CustomerEmailAddress', 'CustomerFullName', 'CustomerDOB', 'City', 'Country']
No. of rows: 82630

In [117]:
raw_df = df

product_count = df.groupBy('ProductId').count()
#product_count = product_count.filter("count >= 3000").orderBy('count', ascending=False) #only counts when the product has 20000 views

raw_df = raw_df.withColumnRenamed("ProductId","product_id_tmp")
raw_df = raw_df.join(product_count, raw_df.product_id_tmp == product_count.ProductId)

user_count = df.groupBy('CustomerId').count()
#user_count = user_count.filter("count >= 20").orderBy('count', ascending=False) #only counts when the user has 100 clicks

raw_df = raw_df.withColumnRenamed("CustomerId","user_id_tmp")
raw_df = raw_df.join(user_count, raw_df.user_id_tmp == user_count.CustomerId)

df = raw_df

df = df.where(df.EntityCode == "Contoso")
#df = df.drop("event_time","category_code","user_session","price","brand","category_id")
df = df.groupBy([df.ProductId, df.CustomerId]).count()
df.columns

StatementMeta(, , , Waiting, )

['ProductId', 'CustomerId', 'count']

In [118]:
from pyspark.sql.types import IntegerType
df = df.withColumn("CustomerId", df["CustomerId"].cast(IntegerType()))
df = df.withColumn("ProductId", df["ProductId"].cast(IntegerType()))
df.printSchema()

StatementMeta(, , , Waiting, )

root
 |-- ProductId: integer (nullable = true)
 |-- CustomerId: integer (nullable = true)
 |-- count: long (nullable = false)

In [119]:
df.show(5)

StatementMeta(, , , Waiting, )

+---------+----------+-----+
|ProductId|CustomerId|count|
+---------+----------+-----+
|      112|       296|   10|
|       96|       296|   10|
|      120|       296|   10|
|      104|       296|   10|
|       88|       296|   10|
+---------+----------+-----+
only showing top 5 rows

In [120]:
#split the data into training and test datatset
train,test=df.randomSplit([0.75,0.25])
print("Training Count:")
train.count()

print("Test Count:")
test.count()

StatementMeta(, , , Waiting, )

Training Count:
Test Count:


2125

#### Alternating least square (ALS) Recommender 

Apache Spark ML implements ALS for collaborative filtering, a very popular algorithm for making recommendations.

ALS recommender is a matrix factorization algorithm that uses Alternating Least Squares with Weighted-Lamda-Regularization (ALS-WR). It factors the user to item matrix A into the user-to-feature matrix U and the item-to-feature matrix M: It runs the ALS algorithm in a parallel fashion. The ALS algorithm should uncover the latent factors that explain the observed user to item ratings and tries to find optimal factor weights to minimize the least squares between predicted and actual ratings.

In [121]:
#import ALS recommender function from pyspark ml library
from pyspark.ml.recommendation import ALS

#Training the recommender model using train datatset
rec = ALS(maxIter=10,regParam=0.20,implicitPrefs = True, userCol='CustomerId',itemCol='ProductId',
            ratingCol='count',nonnegative=True,coldStartStrategy="drop", rank=25)

#fit the model on train set
rec_model = rec.fit(train)

#making predictions on test set 
predicted_ratings = rec_model.transform(test)

#columns in predicted ratings dataframe
predicted_ratings.printSchema()

StatementMeta(, , , Waiting, )

root
 |-- ProductId: integer (nullable = true)
 |-- CustomerId: integer (nullable = true)
 |-- count: long (nullable = false)
 |-- prediction: float (nullable = false)

In [122]:
#predicted vs actual ratings for test set 
predicted_ratings.orderBy(rand()).show(10)

StatementMeta(, , , Waiting, )

+---------+----------+-----+----------+
|ProductId|CustomerId|count|prediction|
+---------+----------+-----+----------+
|       10|        82|   10|0.73039913|
|       81|       425|   10| 0.8356221|
|       76|       676|   10| 0.8442588|
|      109|       109|   10|0.41283312|
|       37|        69|   10| 1.0379064|
|      104|       744|   10|0.73062134|
|       72|       504|   10| 0.6805226|
|       43|       203|   10|0.98426527|
|       43|       835|   10| 0.5193456|
|      110|       998|   10| 0.9319474|
+---------+----------+-----+----------+
only showing top 10 rows

In [123]:
predicted_ratings_witherr=predicted_ratings.withColumn('err',abs(predicted_ratings["prediction"] - predicted_ratings["count"]))
predicted_ratings_witherr.show(5)

StatementMeta(, , , Waiting, )

+---------+----------+-----+----------+--------+
|ProductId|CustomerId|count|prediction|     err|
+---------+----------+-----+----------+--------+
|       31|       375|   10| 0.9148947|9.085105|
|       31|       359|   10| 0.7835939|9.216406|
|       31|       479|   10|0.97031397|9.029686|
|       31|       647|   10|  0.939265|9.060735|
|       31|       191|   10|0.97031397|9.029686|
+---------+----------+-----+----------+--------+
only showing top 5 rows

In [124]:
df.groupBy('count').count().orderBy('count',ascending=True).show(5)

StatementMeta(, , , Waiting, )

+-----+-----+
|count|count|
+-----+-----+
|   10| 8263|
+-----+-----+

In [125]:
predicted_ratings_witherr.groupBy('count').agg({'err':'mean'}).orderBy('count',ascending=True).show(5)

StatementMeta(, , , Waiting, )

+-----+----------------+
|count|        avg(err)|
+-----+----------------+
|   10|9.20190873224595|
+-----+----------------+

In [126]:
#importing Regression Evaluator to measure RMSE
from pyspark.ml.evaluation import RegressionEvaluator

#create Regressor evaluator object for measuring accuracy
evaluator=RegressionEvaluator(metricName='rmse',predictionCol='prediction',labelCol='count')

#apply the RE on predictions dataframe to calculate RMSE
rmse=evaluator.evaluate(predicted_ratings)
print(rmse)

StatementMeta(, , , Waiting, )

9.204261970825783

In [127]:
# Save the model
rec_model.write().overwrite().save("retailai_recommendation_model")

StatementMeta(, , , Waiting, )

###  Below modules can be used to compute recommendations for targeted customers from the above trained ALS model


In [128]:
# loading above trained recommendation model
from pyspark.ml.recommendation import ALSModel
model_reload = ALSModel.load("retailai_recommendation_model")
print(model_reload)

StatementMeta(, , , Waiting, )

ALS_2751ffc156c6

An item-based recommender model that computes cosine similarity for each item pairs using the item factors matrix generated by Spark MLlib's ALS algorithm and recommends top 5 products based on the selected customers.



In [129]:
from pyspark.sql import Row
from pyspark.sql.functions import col
import numpy as np
from numpy import linalg as LA
from pyspark.ml.recommendation import ALSModel

class ItemBasedRecommender():

    def __init__(self, model, spark):
        self.model = model
        self.spark = spark
        self.itemFactors = self.model.itemFactors

    def compute_similarity(self, item_id):
        item = self.itemFactors.where(
            col('id') == item_id).select(col('features'))
        item_features = item.rdd.map(lambda x: x.features).first()

        lol = []
        for row in self.itemFactors.rdd.toLocalIterator():
            _id = row.__getattr__('id')
            features = row.__getattr__('features')
            similarity_score = self._cosine_similarity(features, item_features)
            if _id != item_id:
                lol.append([_id, similarity_score])

        R = Row('item_index', 'similarity_score')
        self.similar_items_df = self.spark.createDataFrame(
            [R(col[0], float(col[1])) for col in lol])
        self.similar_items_df = self.similar_items_df.orderBy(
            col('similarity_score').desc()).na.drop()
        return self.similar_items_df


    def _cosine_similarity(self, vector_1, vector_2):
        v1 = np.asarray(vector_1)
        v2 = np.asarray(vector_2)
        cs = v1.dot(v2) / (LA.norm(v1) * LA.norm(v2))
        return(cs)

StatementMeta(, , , Waiting, )

### Input the CustomerIDs of the millennials to get the top 5 recommended products

In [130]:
# CustomerID as input. Note: this is the normalized CustomerID starting from 0.        
CustomerID = 24
item_based_rec = ItemBasedRecommender(model_reload, spark)
ret_df = item_based_rec.compute_similarity(CustomerID)

ret_df.show(5)

StatementMeta(, , , Waiting, )

+----------+------------------+
|item_index|  similarity_score|
+----------+------------------+
|        32|0.9874732797016829|
|        48|0.9762596620788582|
|        40|0.9738217592468923|
|        56|0.9552652057133719|
|        16|0.8694494686183253|
+----------+------------------+
only showing top 5 rows

#### For WideWorldImporters, top 5 recommended products


In [131]:
df_product_wwi = df_product[df_product["EntityCode"] == 'WideWorldImporters']
rec_df = df_product_wwi.join(ret_df, (df_product_wwi["ProductId"]==ret_df["item_index"]))

rec_df[[ 'ProductCategories', 'ProductId', 'BrandName', 'EntityCode', "similarity_score"]].show(5)

StatementMeta(, , , Waiting, )

+-----------------+---------+---------+----------+----------------+
|ProductCategories|ProductId|BrandName|EntityCode|similarity_score|
+-----------------+---------+---------+----------+----------------+
+-----------------+---------+---------+----------+----------------+